In [7]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os
import zipfile
import pickle
import glob
import pandas as pd
import numpy as np
import gc
import warnings
warnings.filterwarnings('ignore')

In [125]:
# 2. Navigate to your project
os.listdir('/content/drive/MyDrive/')

['Colab Notebooks',
 'h-and-m-personalized-fashion-recommendations',
 'human+activity+recognition+using+smartphones',
 'Memory machines',
 'LLM_Project.gdoc',
 'Dataset']

In [126]:
project_path = '/content/drive/MyDrive/Dataset/'
raw_data_path = f'{project_path}raw_data/'

###Extract one quarter

In [127]:
#Extracting one quarter
# First, let's see what zip files you have
print("Files in Dataset folder:")
print(os.listdir(project_path))

# Extract Q1 2024 first (to explore and build pipeline)
zip_file_path = f'{project_path}faers_ascii_2024q1.zip'

# Create a folder to extract Q1 data
extract_path = '/content/2024_Q1/'
os.makedirs(extract_path, exist_ok=True)

# Extract the zip file
print(f"\nExtracting {zip_file_path}...")
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    print("Extraction complete!")

# See what files were extracted
print(f"\nExtracted files in {extract_path}:")
for root, dirs, files in os.walk(extract_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_size = os.path.getsize(file_path) / (1024*1024)  # Size in MB
        print(f"  {file}: {file_size:.2f} MB")

Files in Dataset folder:
['faers_ascii_2024Q4.zip', 'faers_ascii_2024q2.zip', 'faers_ascii_2024q3.zip', 'faers_ascii_2024q1.zip', 'processed_quarters', 'faers_ascii_2025q2.zip', 'faers_ascii_2025q1.zip', 'faers_ascii_2025q3.zip']

Extracting /content/drive/MyDrive/Dataset/faers_ascii_2024q1.zip...
Extraction complete!

Extracted files in /content/2024_Q1/:
  Readme.pdf: 0.13 MB
  FAQs.pdf: 0.22 MB
  RPSR24Q1.pdf: 0.14 MB
  INDI24Q1.txt: 55.38 MB
  INDI24Q1.pdf: 0.13 MB
  OUTC24Q1.pdf: 0.13 MB
  OUTC24Q1.txt: 6.20 MB
  DEMO24Q1.pdf: 0.21 MB
  THER24Q1.pdf: 0.13 MB
  THER24Q1.txt: 20.61 MB
  REAC24Q1.pdf: 0.13 MB
  DRUG24Q1.pdf: 0.30 MB
  DEMO24Q1.txt: 57.38 MB
  REAC24Q1.txt: 53.39 MB
  RPSR24Q1.txt: 0.26 MB
  DRUG24Q1.txt: 186.83 MB
  ASC_NTS.pdf: 0.33 MB
  DELETE24Q1.txt: 0.04 MB


###Locating extracted ASCII files

In [128]:
# Check if there's a 2024_Q1 folder (from our extraction)
if '2024_Q1' in os.listdir('/content/'):
    print("Contents of /content/2024_Q1/:")
    print(os.listdir('/content/2024_Q1/'))

    # Check for subdirectories
    for item in os.listdir('/content/2024_Q1/'):
        item_path = f'/content/2024_Q1/{item}'
        if os.path.isdir(item_path):
            print(f"\nContents of {item_path}:")
            files = os.listdir(item_path)
            # Show first 10 files
            for f in files[:10]:
                print(f"  - {f}")

Contents of /content/2024_Q1/:
['Readme.pdf', 'ASCII', 'FAQs.pdf', 'Deleted']

Contents of /content/2024_Q1/ASCII:
  - RPSR24Q1.pdf
  - INDI24Q1.txt
  - INDI24Q1.pdf
  - OUTC24Q1.pdf
  - OUTC24Q1.txt
  - DEMO24Q1.pdf
  - THER24Q1.pdf
  - THER24Q1.txt
  - REAC24Q1.pdf
  - DRUG24Q1.pdf

Contents of /content/2024_Q1/Deleted:
  - DELETE24Q1.txt


###Exploe file sttructure

In [129]:
# Set the correct path to your ASCII files
ascii_folder = '/content/2024_Q1/ASCII/'

In [130]:
# Get list of only .txt files (excluding PDFs)
txt_files = [f for f in os.listdir(ascii_folder) if f.endswith('.txt')]
print(f"Found {len(txt_files)} text files:")
for f in sorted(txt_files):
    size_mb = os.path.getsize(os.path.join(ascii_folder, f)) / (1024*1024)
    print(f"  {f}: {size_mb:.2f} MB")

Found 7 text files:
  DEMO24Q1.txt: 57.38 MB
  DRUG24Q1.txt: 186.83 MB
  INDI24Q1.txt: 55.38 MB
  OUTC24Q1.txt: 6.20 MB
  REAC24Q1.txt: 53.39 MB
  RPSR24Q1.txt: 0.26 MB
  THER24Q1.txt: 20.61 MB


In [131]:
# Define the core files for your project
Ascii_files = {
    'DEMO': 'DEMO24Q1.txt',
    'DRUG': 'DRUG24Q1.txt',
    'REAC': 'REAC24Q1.txt',
    'OUTC': 'OUTC24Q1.txt',
    'INDI': 'INDI24Q1.txt',
    'THER': 'THER24Q1.txt',
    'RSPR': 'RPSR24Q1.txt'
}

# Check each file's structure
for file_type, filename in Ascii_files.items():
    if filename in txt_files:
        file_path = os.path.join(ascii_folder, filename)

        # Read first 3 rows to see structure
        df_sample = pd.read_csv(file_path, sep='$', nrows=3, encoding='latin-1', low_memory=False)

        print(f"\n{file_type} File: {filename}")
        print(f"Columns ({len(df_sample.columns)}): {list(df_sample.columns)}")
        print("\nFirst 2 rows:")
        print(df_sample.head(2))


DEMO File: DEMO24Q1.txt
Columns (25): ['primaryid', 'caseid', 'caseversion', 'i_f_code', 'event_dt', 'mfr_dt', 'init_fda_dt', 'fda_dt', 'rept_cod', 'auth_num', 'mfr_num', 'mfr_sndr', 'lit_ref', 'age', 'age_cod', 'age_grp', 'sex', 'e_sub', 'wt', 'wt_cod', 'rept_dt', 'to_mfr', 'occp_cod', 'reporter_country', 'occr_country']

First 2 rows:
    primaryid    caseid  caseversion i_f_code    event_dt    mfr_dt  \
0  1001678125  10016781           25        F  20120330.0  20240125   
1  1002872124  10028721           24        F  20171025.0  20240228   

   init_fda_dt    fda_dt rept_cod  auth_num  ... age_grp sex e_sub  wt wt_cod  \
0     20140318  20240129      EXP       NaN  ...     NaN   F     Y NaN    NaN   
1     20140321  20240304      EXP       NaN  ...     NaN   F     Y NaN    NaN   

    rept_dt to_mfr occp_cod  reporter_country  occr_country  
0  20240129    NaN       HP                CA            CA  
1  20240304    NaN       MD                CA            CA  

[2 rows x 25 co

###Initial Data Inspection & Quality Check

In [132]:
#Load the dataset
# 1. Load DEMO (Demographics)
print("1. Loading DEMO...")
demo = pd.read_csv(f'{ascii_folder}DEMO24Q1.txt', sep='$', encoding='latin-1', low_memory=False)
print(f"   ✓ DEMO loaded: {demo.shape[0]:,} records, {demo.shape[1]} columns")

# 2. Load DRUG (Drug Information)
print("\n2. Loading DRUG...")
drug = pd.read_csv(f'{ascii_folder}DRUG24Q1.txt', sep='$', encoding='latin-1', low_memory=False)
print(f"   ✓ DRUG loaded: {drug.shape[0]:,} records")

# 3. Load REAC (Reactions/Adverse Events)
print("\n3. Loading REAC...")
reac = pd.read_csv(f'{ascii_folder}REAC24Q1.txt', sep='$', encoding='latin-1', low_memory=False)
print(f"   ✓ REAC loaded: {reac.shape[0]:,} records")

# 4. Load OUTC (Outcomes)
print("\n4. Loading OUTC...")
outc = pd.read_csv(f'{ascii_folder}OUTC24Q1.txt', sep='$', encoding='latin-1', low_memory=False)
print(f"   ✓ OUTC loaded: {outc.shape[0]:,} records")

# 5. Load INDI (Indications)
print("\n5. Loading INDI...")
indi = pd.read_csv(f'{ascii_folder}INDI24Q1.txt', sep='$', encoding='latin-1', low_memory=False)
print(f"   ✓ INDI loaded: {indi.shape[0]:,} records")

1. Loading DEMO...
   ✓ DEMO loaded: 406,184 records, 25 columns

2. Loading DRUG...
   ✓ DRUG loaded: 1,909,327 records

3. Loading REAC...
   ✓ REAC loaded: 1,445,416 records

4. Loading OUTC...
   ✓ OUTC loaded: 295,044 records

5. Loading INDI...
   ✓ INDI loaded: 1,186,115 records


In [133]:
# Check for missing primaryid (critical for joining)
print("Missing Primary IDs:")
print(f"  DEMO: {demo['primaryid'].isna().sum()}")
print(f"  DRUG: {drug['primaryid'].isna().sum()}")
print(f"  REAC: {reac['primaryid'].isna().sum()}")
print(f"  OUTC: {outc['primaryid'].isna().sum()}")
print(f"  INDI: {indi['primaryid'].isna().sum()}")

Missing Primary IDs:
  DEMO: 0
  DRUG: 0
  REAC: 0
  OUTC: 0
  INDI: 0


In [134]:
# Check for duplicates
print("\nDuplicate Records Check:")
print(f"  DEMO duplicate primaryids: {demo['primaryid'].duplicated().sum()}")
print(f"  DRUG duplicate (primaryid, drug_seq): {drug[['primaryid', 'drug_seq']].duplicated().sum()}")
print(f"  REAC duplicate (primaryid, pt): {reac[['primaryid', 'pt']].duplicated().sum()}")
#OUTC: Multiple outcomes per patient is NORMAL
#INDI: Multiple indications per patient is NORMAL


Duplicate Records Check:
  DEMO duplicate primaryids: 0
  DRUG duplicate (primaryid, drug_seq): 92
  REAC duplicate (primaryid, pt): 21934


In [135]:
# 1. Handle DRUG duplicates
print("1. Cleaning DRUG duplicates...")
print(f"   Before: {drug.shape[0]:,} records")

# Check what kind of duplicates these are
dup_drugs = drug[drug[['primaryid', 'drug_seq']].duplicated(keep=False)]
print(f"   Sample duplicate drug entries:")
sample_dup = dup_drugs.head(10)[['primaryid', 'drug_seq', 'drugname', 'role_cod']]
print(sample_dup)

# Remove exact duplicates (keeping first occurrence)
drug_clean = drug.drop_duplicates(subset=['primaryid', 'drug_seq'], keep='first')
print(f"   After: {drug_clean.shape[0]:,} records")
print(f"   Removed: {drug.shape[0] - drug_clean.shape[0]} duplicate drug records")

1. Cleaning DRUG duplicates...
   Before: 1,909,327 records
   Sample duplicate drug entries:
        primaryid  drug_seq    drugname role_cod
809486  233626421         1  TACROLIMUS       PS
809487  233626421         1  TACROLIMUS       PS
809506  233626461         1       TALTZ       PS
809507  233626461         1       TALTZ       PS
809511  233626481         1       TALTZ       PS
809512  233626481         1       TALTZ       PS
809513  233626491         1       TALTZ       PS
809514  233626491         1       TALTZ       PS
810176  233628921         1     MENOPUR       PS
810177  233628921         1     MENOPUR       PS
   After: 1,909,235 records
   Removed: 92 duplicate drug records


In [136]:
# 2. Handle REAC duplicates
print("\n2. Cleaning REAC duplicates...")
print(f"   Before: {reac.shape[0]:,} records")

# Check what kind of duplicates these are
dup_reacs = reac[reac[['primaryid', 'pt']].duplicated(keep=False)]
print(f"   Sample duplicate reaction entries:")
sample_dup_reac = dup_reacs.head(10)[['primaryid', 'pt']]
print(sample_dup_reac)

# For reactions, duplicates might mean the same reaction reported multiple times
# We should keep only unique primaryid-reaction pairs
reac_clean = reac.drop_duplicates(subset=['primaryid', 'pt'], keep='first')
print(f"   After: {reac_clean.shape[0]:,} records")
print(f"   Removed: {reac.shape[0] - reac_clean.shape[0]} duplicate reaction records")

# 3. Verify no duplicates remain
print("\n3. Verifying cleanup...")
print(f"   DRUG duplicates remaining: {drug_clean[['primaryid', 'drug_seq']].duplicated().sum()}")
print(f"   REAC duplicates remaining: {reac_clean[['primaryid', 'pt']].duplicated().sum()}")


2. Cleaning REAC duplicates...
   Before: 1,445,416 records
   Sample duplicate reaction entries:
      primaryid                              pt
11   1001678125             Injection site pain
14   1001678125             Injection site pain
206  1014222252                     Head injury
211  1014222252              Hypoaesthesia oral
213  1014222252                     Head injury
216  1014222252              Hypoaesthesia oral
234  1014222252           Loss of consciousness
235  1014222252           Loss of consciousness
292  1016133060  Intraocular pressure increased
311  1016133060  Intraocular pressure increased
   After: 1,423,482 records
   Removed: 21934 duplicate reaction records

3. Verifying cleanup...
   DRUG duplicates remaining: 0
   REAC duplicates remaining: 0


In [137]:
# 4. Update the dataframes for further processing
drug = drug_clean.copy()
reac = reac_clean.copy()

In [138]:
# Find common primaryids across files
print("\nOverlap Analysis:")
demo_ids = set(demo['primaryid'].unique())
drug_ids = set(drug['primaryid'].unique())
reac_ids = set(reac['primaryid'].unique())
outc_ids = set(outc['primaryid'].unique())
indi_ids = set(indi['primaryid'].unique())

common_ids = demo_ids.intersection(drug_ids).intersection(reac_ids)

print(f"  Total unique reports in DEMO: {len(demo_ids):,}")
print(f"  Reports with drugs: {len(drug_ids):,}")
print(f"  Reports with reactions: {len(reac_ids):,}")
print(f"  Reports with outcomes: {len(outc_ids):,}")
print(f"  Reports with indications: {len(indi_ids):,}")
print(f"  Common reports (DEMO ∩ DRUG ∩ REAC): {len(common_ids):,}")


Overlap Analysis:
  Total unique reports in DEMO: 406,184
  Reports with drugs: 406,184
  Reports with reactions: 406,184
  Reports with outcomes: 222,364
  Reports with indications: 380,226
  Common reports (DEMO ∩ DRUG ∩ REAC): 406,184


In [139]:
# Check drug role distribution
print("\nDrug Role Distribution:")
print(drug['role_cod'].value_counts())


Drug Role Distribution:
role_cod
SS    819688
C     672902
PS    406184
I      10461
Name: count, dtype: int64


###Data Preprocessing

In [140]:
# Understanding role codes:
role_meanings = {
    'PS': 'Primary Suspect - Drug suspected to cause the adverse event',
    'SS': 'Secondary Suspect - Other drugs also suspected',
    'C': 'Concomitant - Drug taken but not suspected',
    'I': 'Interacting - Drug that may interact with suspect drugs'
}

print("Drug Role Definitions:")
for code, meaning in role_meanings.items():
    count = drug['role_cod'].value_counts().get(code, 0)
    pct = (count / len(drug)) * 100
    print(f"  {code}: {count:,} ({pct:.1f}%) - {meaning}")

Drug Role Definitions:
  PS: 406,184 (21.3%) - Primary Suspect - Drug suspected to cause the adverse event
  SS: 819,688 (42.9%) - Secondary Suspect - Other drugs also suspected
  C: 672,902 (35.2%) - Concomitant - Drug taken but not suspected
  I: 10,461 (0.5%) - Interacting - Drug that may interact with suspect drugs


In [141]:
# 1. Suspect drugs only (PS + SS + I) - Most important for adverse event associations
suspect_drugs = drug[drug['role_cod'].isin(['PS', 'SS', 'I'])].copy()
print(f"\n1. SUSPECT DRUGS (PS+SS+I): {suspect_drugs.shape[0]:,} records")
print(f"   These are drugs suspected to cause adverse events")

# 2. Primary suspects only - Highest confidence drug-event associations
primary_drugs = drug[drug['role_cod'] == 'PS'].copy()
print(f"\n2. PRIMARY SUSPECTS ONLY: {primary_drugs.shape[0]:,} records")
print(f"   These have the strongest association with adverse events")

# 3. All drugs including concomitant - For drug interaction analysis
all_drugs = drug.copy()
print(f"\n3. ALL DRUGS (including concomitant): {all_drugs.shape[0]:,} records")
print(f"   Use this for drug-drug interaction discovery")

# Check how many patients have each type of drug
print("\n" + "-"*40)
print("Patients by drug type:")
print(f"  Patients with Primary Suspect drugs: {primary_drugs['primaryid'].nunique():,}")
print(f"  Patients with Secondary Suspect drugs: {drug[drug['role_cod']=='SS']['primaryid'].nunique():,}")
print(f"  Patients with Concomitant drugs: {drug[drug['role_cod']=='C']['primaryid'].nunique():,}")
print(f"  Patients with Interacting drugs: {drug[drug['role_cod']=='I']['primaryid'].nunique():,}")


1. SUSPECT DRUGS (PS+SS+I): 1,236,333 records
   These are drugs suspected to cause adverse events

2. PRIMARY SUSPECTS ONLY: 406,184 records
   These have the strongest association with adverse events

3. ALL DRUGS (including concomitant): 1,909,235 records
   Use this for drug-drug interaction discovery

----------------------------------------
Patients by drug type:
  Patients with Primary Suspect drugs: 406,184
  Patients with Secondary Suspect drugs: 172,663
  Patients with Concomitant drugs: 109,972
  Patients with Interacting drugs: 2,327


In [142]:
# Analyze polypharmacy by role
print("Drug combinations analysis:")
drug_counts = drug.groupby(['primaryid', 'role_cod']).size().unstack(fill_value=0)
print(f"  Avg Primary Suspects per patient: {drug_counts['PS'].mean():.2f}")
print(f"  Avg Secondary Suspects per patient: {drug_counts['SS'].mean():.2f}")
print(f"  Avg Concomitant drugs per patient: {drug_counts['C'].mean():.2f}")

# Patients with multiple suspects (potential drug interactions)
multi_suspect = drug_counts[(drug_counts['PS'] > 0) & (drug_counts['SS'] > 0)]
print(f"  Patients with both PS and SS drugs: {len(multi_suspect):,} (potential interactions)")

Drug combinations analysis:
  Avg Primary Suspects per patient: 1.00
  Avg Secondary Suspects per patient: 2.02
  Avg Concomitant drugs per patient: 1.66
  Patients with both PS and SS drugs: 172,663 (potential interactions)


## DRUG DATASET

In [143]:
## Step 1: Create focused drug datasets

# A. Main dataset - Suspect drugs only
suspect_drugs = drug[drug['role_cod'].isin(['PS', 'SS', 'I'])].copy()
suspect_drugs['drugname_clean'] = suspect_drugs['drugname'].str.upper().str.strip()

# Remove invalid drug names
invalid_names = ['UNKNOWN', 'UNK', 'NULL', 'NA', 'N/A', 'NOT SPECIFIED', 'UNSPECIFIED', 'BLINDED']
suspect_drugs = suspect_drugs[~suspect_drugs['drugname_clean'].isin(invalid_names)]
print(f"✓ Suspect drugs cleaned: {suspect_drugs.shape[0]:,} records")

# B. Concomitant drugs - Keep separately for interaction discovery
concomitant_drugs = drug[drug['role_cod'] == 'C'].copy()
concomitant_drugs['drugname_clean'] = concomitant_drugs['drugname'].str.upper().str.strip()
concomitant_drugs = concomitant_drugs[~concomitant_drugs['drugname_clean'].isin(invalid_names)]
print(f"✓ Concomitant drugs cleaned: {concomitant_drugs.shape[0]:,} records")


✓ Suspect drugs cleaned: 1,236,333 records
✓ Concomitant drugs cleaned: 672,902 records


In [144]:
# Step 2: Create patient-level drug profiles
print("Step 2: Creating patient drug profiles...")

# Get suspect drug combinations per patient
patient_suspect_drugs = suspect_drugs.groupby('primaryid').agg({
    'drugname_clean': lambda x: list(set(x)),
    'role_cod': lambda x: dict(x.value_counts())
}).rename(columns={'drugname_clean': 'suspect_drugs', 'role_cod': 'role_distribution'})

patient_suspect_drugs['num_suspect_drugs'] = patient_suspect_drugs['suspect_drugs'].apply(len)

# Get concomitant drugs per patient
patient_concom_drugs = concomitant_drugs.groupby('primaryid')['drugname_clean'].apply(lambda x: list(set(x))).to_frame()
patient_concom_drugs.columns = ['concomitant_drugs']
patient_concom_drugs['num_concom_drugs'] = patient_concom_drugs['concomitant_drugs'].apply(len)

print(f"Drug profiles created for {len(patient_suspect_drugs):,} patients")

patient_suspect_drugs['is_interaction_case'] = (patient_suspect_drugs['num_suspect_drugs'] > 1).astype(int)     ##Flag for cases with multiple suspect drugs

primary_drug = suspect_drugs[suspect_drugs['role_cod'] == 'PS'].groupby('primaryid')['drugname_clean'].first() ##The main PS drug (every case has exactly one)
patient_suspect_drugs['primary_drug'] = patient_suspect_drugs.index.map(primary_drug)


Step 2: Creating patient drug profiles...
Drug profiles created for 406,184 patients


In [145]:
# Step 3: Identify key patterns
print("\nStep 3: Analyzing drug patterns...")

# Polypharmacy categories
patient_suspect_drugs['polypharmacy_category'] = pd.cut(
    patient_suspect_drugs['num_suspect_drugs'],
    bins=[0, 1, 2, 5, 10, 100],
    labels=['monotherapy', 'dual_therapy', 'moderate_poly', 'severe_poly', 'extreme_poly']
)

print("Polypharmacy distribution (suspect drugs):")
print(patient_suspect_drugs['polypharmacy_category'].value_counts())

# Top drug combinations (for PS+SS patients)
multi_drug_patients = patient_suspect_drugs[patient_suspect_drugs['num_suspect_drugs'] > 1]
print(f"\n{len(multi_drug_patients):,} patients on multiple suspect drugs")

# Find most common drug pairs
print("\nFinding common drug combinations...")
drug_pairs = []
for drugs in multi_drug_patients['suspect_drugs'].head(10000):  # Sample for speed
    if len(drugs) >= 2:
        for i in range(len(drugs)):
            for j in range(i+1, len(drugs)):
                drug_pairs.append(tuple(sorted([drugs[i], drugs[j]])))

from collections import Counter
pair_counts = Counter(drug_pairs)
print("\nTop 30 drug pairs (potential interactions):")
print("="*60)
for idx, (pair, count) in enumerate(pair_counts.most_common(30), 1):
    print(f"  {idx:2d}. {pair[0]} + {pair[1]}: {count} cases")


Step 3: Analyzing drug patterns...
Polypharmacy distribution (suspect drugs):
polypharmacy_category
monotherapy      329750
dual_therapy      36906
moderate_poly     27477
severe_poly        7873
extreme_poly       4141
Name: count, dtype: int64

76,434 patients on multiple suspect drugs

Finding common drug combinations...

Top 30 drug pairs (potential interactions):
   1. PREDNISONE + RITUXIMAB: 252 cases
   2. METHOTREXATE + RITUXIMAB: 207 cases
   3. ACTEMRA + RITUXIMAB: 177 cases
   4. METHOTREXATE + PREDNISONE: 174 cases
   5. CYCLOPHOSPHAMIDE + RITUXIMAB: 167 cases
   6. COSENTYX + METHOTREXATE: 162 cases
   7. RISPERDAL + RISPERIDONE: 158 cases
   8. LEFLUNOMIDE + METHOTREXATE: 155 cases
   9. OPDIVO + YERVOY: 154 cases
  10. HUMIRA + METHOTREXATE: 150 cases
  11. ENBREL + HUMIRA: 147 cases
  12. ACTEMRA + METHOTREXATE: 146 cases
  13. ATRIPLA + TRUVADA: 142 cases
  14. HUMIRA + REMICADE: 136 cases
  15. ENBREL + METHOTREXATE: 132 cases
  16. ENBREL + REMICADE: 128 cases
  17.

##Demographics Data

In [146]:
# Create clean demographics dataframe
demo_clean = demo.copy()

In [147]:
# 1. Standardize age to years
print("1. Standardizing age to years...")
age_conversion = {
    'YR': 1,           # Years
    'MON': 1/12,       # Months to years
    'WK': 1/52,        # Weeks to years
    'DY': 1/365,       # Days to years
    'DEC': 10,         # Decades to years
    'HR': 1/(365*24),  # Hours to years (for newborns)
    'YRS': 1           # Alternative years notation
}

# Create age_years column
demo_clean['age_years'] = np.nan
for unit, multiplier in age_conversion.items():
    mask = demo_clean['age_cod'] == unit
    if mask.any():
        demo_clean.loc[mask, 'age_years'] = pd.to_numeric(demo_clean.loc[mask, 'age'], errors='coerce') * multiplier
        print(f"  Converted {mask.sum():,} ages from {unit}")

1. Standardizing age to years...
  Converted 241,545 ages from YR
  Converted 1,467 ages from MON
  Converted 141 ages from WK
  Converted 1,657 ages from DY
  Converted 3,591 ages from DEC
  Converted 26 ages from HR


In [148]:
# 2. Create age groups for association rules
print("\n2. Creating age groups for mining...")
demo_clean['age_group'] = pd.cut(
    demo_clean['age_years'],
    bins=[0, 2, 12, 18, 45, 65, 80, 150],
    labels=['infant', 'child', 'adolescent', 'adult', 'middle_age', 'elderly', 'very_elderly'],
    include_lowest=True
)


2. Creating age groups for mining...


In [149]:
# 3. Clean sex values
print("\n3. Cleaning sex values...")
sex_mapping = {'M': 'Male', 'F': 'Female', 'UNK': 'Unknown', 'U': 'Unknown', np.nan: 'Unknown'}
demo_clean['sex_clean'] = demo_clean['sex'].map(sex_mapping).fillna('Unknown')

# 4. Country analysis
print("\n4. Analyzing reporter countries...")
top_countries = demo_clean['occr_country'].value_counts().head(10)
print("Top 10 countries by reports:")
for country, count in top_countries.items():
    pct = (count/len(demo_clean))*100
    print(f"  {country}: {count:,} ({pct:.1f}%)")


3. Cleaning sex values...

4. Analyzing reporter countries...
Top 10 countries by reports:
  US: 242,903 (59.8%)
  CA: 26,980 (6.6%)
  JP: 14,876 (3.7%)
  GB: 13,469 (3.3%)
  FR: 13,446 (3.3%)
  DE: 7,454 (1.8%)
  CN: 5,731 (1.4%)
  IT: 5,218 (1.3%)
  ES: 4,076 (1.0%)
  AU: 3,384 (0.8%)


In [150]:
# 5. Create final demographics features
demo_processed = demo_clean[['primaryid', 'age_years', 'age_group', 'sex_clean', 'occr_country']].copy()

# Add binary features for association mining
demo_processed['is_elderly'] = (demo_processed['age_years'] >= 65).astype(int)
demo_processed['is_pediatric'] = (demo_processed['age_years'] < 18).astype(int)
#demo_processed['is_female'] = (demo_processed['sex_clean'] == 'Female').astype(int) -- Removing it as it is redundant. we have sex_clean column for that

# Summary statistics
print("\n" + "-"*40)
print("Demographics Summary:")
print(f"  Total patients: {len(demo_processed):,}")
print(f"  Age available: {demo_processed['age_years'].notna().sum():,} ({(demo_processed['age_years'].notna().sum()/len(demo_processed)*100):.1f}%)")
print(f"  Mean age: {demo_processed['age_years'].mean():.1f} years")
print(f"  Median age: {demo_processed['age_years'].median():.1f} years")
print(f"  Elderly (≥65): {demo_processed['is_elderly'].sum():,} ({(demo_processed['is_elderly'].sum()/len(demo_processed)*100):.1f}%)")
print(f"  Pediatric (<18): {demo_processed['is_pediatric'].sum():,} ({(demo_processed['is_pediatric'].sum()/len(demo_processed)*100):.1f}%)")

print("\nAge group distribution:")
age_dist = demo_processed['age_group'].value_counts()
for group, count in age_dist.items():
    pct = (count/len(demo_processed))*100
    print(f"  {group}: {count:,} ({pct:.1f}%)")

print("\nSex distribution:")
sex_dist = demo_processed['sex_clean'].value_counts()
for sex, count in sex_dist.items():
    pct = (count/len(demo_processed))*100
    print(f"  {sex}: {count:,} ({pct:.1f}%)")


----------------------------------------
Demographics Summary:
  Total patients: 406,184
  Age available: 248,412 (61.2%)
  Mean age: 55.2 years
  Median age: 59.0 years
  Elderly (≥65): 98,580 (24.3%)
  Pediatric (<18): 17,897 (4.4%)

Age group distribution:
  middle_age: 82,439 (20.3%)
  elderly: 73,085 (18.0%)
  adult: 53,814 (13.2%)
  very_elderly: 20,034 (4.9%)
  child: 8,347 (2.1%)
  adolescent: 7,820 (1.9%)
  infant: 2,870 (0.7%)

Sex distribution:
  Female: 204,178 (50.3%)
  Male: 137,899 (33.9%)
  Unknown: 64,107 (15.8%)


##Reaction Dataset

In [151]:
# Clean reactions (we already removed duplicates earlier)
reac_clean = reac.copy()

In [152]:
# 1. Standardize reaction terms
print("1. Standardizing reaction terms...")
reac_clean['pt_clean'] = reac_clean['pt'].str.upper().str.strip()

# Remove invalid/uninformative reactions
invalid_reactions = ['UNKNOWN', 'NA', 'NONE', 'NO ADVERSE EVENT', 'NO ADVERSE DRUG REACTION']
before_count = len(reac_clean)
reac_clean = reac_clean[~reac_clean['pt_clean'].isin(invalid_reactions)]
print(f"  Removed {before_count - len(reac_clean):,} invalid reaction records")

1. Standardizing reaction terms...
  Removed 5,484 invalid reaction records


In [153]:
# 2. Identify serious reactions based on MedDRA preferred terms
print("\n2. Identifying serious adverse events...")
serious_keywords = [
    'DEATH', 'DIED', 'FATAL',
    'CARDIAC ARREST', 'MYOCARDIAL INFARCTION', 'HEART FAILURE',
    'STROKE', 'CEREBROVASCULAR', 'HAEMORRHAGE',
    'ANAPHYLACTIC', 'STEVENS-JOHNSON', 'TOXIC EPIDERMAL NECROLYSIS',
    'HEPATIC FAILURE', 'LIVER FAILURE', 'HEPATOTOXICITY',
    'RENAL FAILURE', 'KIDNEY FAILURE',
    'RESPIRATORY FAILURE', 'RESPIRATORY ARREST',
    'SEPSIS', 'SEPTIC SHOCK',
    'SUICIDE', 'OVERDOSE'
]

# Flag serious reactions
reac_clean['is_serious'] = reac_clean['pt_clean'].apply(
    lambda x: any(keyword in x for keyword in serious_keywords) if pd.notna(x) else False
)

print(f"  Reactions flagged as serious: {reac_clean['is_serious'].sum():,}")


2. Identifying serious adverse events...
  Reactions flagged as serious: 53,982


In [154]:
# 3. Group reactions by patient
print("\n3. Creating patient reaction profiles...")
patient_reactions = reac_clean.groupby('primaryid').agg({
    'pt_clean': lambda x: list(set(x)),
    'is_serious': 'max'
}).rename(columns={'pt_clean': 'all_reactions', 'is_serious': 'has_serious_reaction'})

patient_reactions['num_reactions'] = patient_reactions['all_reactions'].apply(len)

# Create reaction categories
patient_reactions['reaction_severity'] = pd.cut(
    patient_reactions['num_reactions'],
    bins=[0, 1, 3, 5, 10, 100],
    labels=['single', 'few', 'moderate', 'many', 'extreme']
)


3. Creating patient reaction profiles...


In [155]:
# 4. Top adverse events analysis
print("\n4. Analyzing top adverse events...")
top_reactions = reac_clean['pt_clean'].value_counts().head(30)
print("Top 30 most reported adverse events:")
print("="*60)
for idx, (reaction, count) in enumerate(top_reactions.items(), 1):
    pct = (count/len(reac_clean))*100
    serious_flag = "⚠️ SERIOUS" if any(kw in reaction for kw in serious_keywords) else ""
    print(f"  {idx:2d}. {reaction}: {count:,} ({pct:.2f}%) {serious_flag}")

# 5. Statistics
print("\n" + "-"*40)
print("Reaction Statistics:")
print(f"  Total reaction records: {len(reac_clean):,}")
print(f"  Unique adverse events: {reac_clean['pt_clean'].nunique():,}")
print(f"  Patients with reactions: {len(patient_reactions):,}")
print(f"  Mean reactions per patient: {patient_reactions['num_reactions'].mean():.2f}")
print(f"  Median reactions per patient: {patient_reactions['num_reactions'].median():.0f}")
print(f"  Patients with serious reactions: {patient_reactions['has_serious_reaction'].sum():,} ({(patient_reactions['has_serious_reaction'].sum()/len(patient_reactions)*100):.1f}%)")

print("\nReaction count distribution:")
reaction_dist = patient_reactions['reaction_severity'].value_counts()
for severity, count in reaction_dist.items():
    pct = (count/len(patient_reactions))*100
    print(f"  {severity}: {count:,} ({pct:.1f}%)")



4. Analyzing top adverse events...
Top 30 most reported adverse events:
   1. OFF LABEL USE: 32,627 (2.30%) 
   2. DRUG INEFFECTIVE: 23,762 (1.68%) 
   3. FATIGUE: 20,196 (1.42%) 
   4. NAUSEA: 16,577 (1.17%) 
   5. DIARRHOEA: 16,301 (1.15%) 
   6. PRODUCT DOSE OMISSION ISSUE: 16,030 (1.13%) 
   7. DEATH: 15,906 (1.12%) ⚠️ SERIOUS
   8. COVID-19: 13,034 (0.92%) 
   9. HEADACHE: 12,982 (0.92%) 
  10. PAIN: 12,794 (0.90%) 
  11. DYSPNOEA: 11,612 (0.82%) 
  12. ARTHRALGIA: 11,163 (0.79%) 
  13. PRURITUS: 10,500 (0.74%) 
  14. VOMITING: 9,701 (0.68%) 
  15. RASH: 9,588 (0.68%) 
  16. DRUG DOSE OMISSION BY DEVICE: 9,577 (0.68%) 
  17. INAPPROPRIATE SCHEDULE OF PRODUCT ADMINISTRATION: 9,466 (0.67%) 
  18. DIZZINESS: 9,441 (0.67%) 
  19. MALAISE: 9,420 (0.66%) 
  20. CONDITION AGGRAVATED: 8,934 (0.63%) 
  21. INJECTION SITE PAIN: 8,509 (0.60%) 
  22. ASTHENIA: 8,501 (0.60%) 
  23. PNEUMONIA: 8,171 (0.58%) 
  24. COUGH: 8,140 (0.57%) 
  25. PYREXIA: 8,076 (0.57%) 
  26. INCORRECT DOSE ADMINIS

##Outcomes and Indications Dataset

In [156]:
# PART A: Process Outcomes
outc_clean = outc.copy()

In [157]:
# Map outcome codes to descriptions
outcome_mapping = {
    'DE': 'Death',
    'LT': 'Life-Threatening',
    'HO': 'Hospitalization - Initial or Prolonged',
    'DS': 'Disability',
    'CA': 'Congenital Anomaly',
    'RI': 'Required Intervention to Prevent Permanent Impairment',
    'OT': 'Other Serious (Important Medical Event)'
}
outc_clean['outcome_desc'] = outc_clean['outc_cod'].map(outcome_mapping)



# Define serious outcomes
serious_outcome_codes = ['DE', 'LT', 'HO', 'DS', 'CA']
outc_clean['is_serious_outcome'] = outc_clean['outc_cod'].isin(serious_outcome_codes)

print("Outcome Distribution:")
for code, desc in outcome_mapping.items():
    count = (outc_clean['outc_cod'] == code).sum()
    if count > 0:
        pct = (count/len(outc_clean))*100
        serious = "⚠️ SERIOUS" if code in serious_outcome_codes else ""
        print(f"  {code} ({desc}): {count:,} ({pct:.1f}%) {serious}")

Outcome Distribution:
  DE (Death): 32,303 (10.9%) ⚠️ SERIOUS
  LT (Life-Threatening): 11,990 (4.1%) ⚠️ SERIOUS
  HO (Hospitalization - Initial or Prolonged): 82,534 (28.0%) ⚠️ SERIOUS
  DS (Disability): 6,112 (2.1%) ⚠️ SERIOUS
  CA (Congenital Anomaly): 895 (0.3%) ⚠️ SERIOUS
  RI (Required Intervention to Prevent Permanent Impairment): 981 (0.3%) 
  OT (Other Serious (Important Medical Event)): 160,229 (54.3%) 


In [158]:
# Group by patient
patient_outcomes = outc_clean.groupby('primaryid').agg({
    'outc_cod': lambda x: list(x),
    'is_serious_outcome': 'max',
    'outcome_desc': lambda x: list(x)
}).rename(columns={'outc_cod': 'outcome_codes', 'outcome_desc': 'outcome_descriptions'})

print(f"\nOutcome Statistics:")
print(f"  Total patients with outcomes: {len(patient_outcomes):,}")
print(f"  Patients with serious outcomes: {patient_outcomes['is_serious_outcome'].sum():,} ({(patient_outcomes['is_serious_outcome'].sum()/len(patient_outcomes)*100):.1f}%)")

# Check for death outcomes specifically
death_patients = outc_clean[outc_clean['outc_cod'] == 'DE']['primaryid'].nunique()
print(f"  Patients with death outcome: {death_patients:,} ({(death_patients/406184*100):.2f}% of all patients)")



Outcome Statistics:
  Total patients with outcomes: 222,364
  Patients with serious outcomes: 115,725 (52.0%)
  Patients with death outcome: 32,303 (7.95% of all patients)


In [159]:
# PART B: Process Indications

indi_clean = indi.copy()

In [160]:
# Standardize indication terms
indi_clean['indi_pt_clean'] = indi_clean['indi_pt'].str.upper().str.strip()

# Remove invalid indications
invalid_indications = ['UNKNOWN', 'UNK', 'PRODUCT USED FOR UNKNOWN INDICATION', 'NA', 'NONE']
before_indi = len(indi_clean)
indi_clean = indi_clean[~indi_clean['indi_pt_clean'].isin(invalid_indications)]
print(f"Removed {before_indi - len(indi_clean):,} invalid indication records")

# Group by patient
patient_indications = indi_clean.groupby('primaryid')['indi_pt_clean'].apply(lambda x: list(set(x))).to_frame()
patient_indications.columns = ['indications']
patient_indications['num_indications'] = patient_indications['indications'].apply(len)

Removed 492,799 invalid indication records


In [161]:
# Top indications
print(f"\nIndication Statistics:")
print(f"  Total patients with indications: {len(patient_indications):,}")
print(f"  Mean indications per patient: {patient_indications['num_indications'].mean():.2f}")

top_indications = indi_clean['indi_pt_clean'].value_counts().head(30)
print("\nTop 30 Drug Indications (Why drugs were prescribed):")
print("="*60)
for idx, (indication, count) in enumerate(top_indications.items(), 1):
    pct = (count/len(indi_clean))*100
    print(f"  {idx:2d}. {indication}: {count:,} ({pct:.2f}%)")


Indication Statistics:
  Total patients with indications: 290,535
  Mean indications per patient: 1.44

Top 30 Drug Indications (Why drugs were prescribed):
   1. RHEUMATOID ARTHRITIS: 47,062 (6.79%)
   2. TYPE 2 DIABETES MELLITUS: 19,351 (2.79%)
   3. PLASMA CELL MYELOMA: 16,479 (2.38%)
   4. DERMATITIS ATOPIC: 13,333 (1.92%)
   5. ASTHMA: 13,332 (1.92%)
   6. CROHN'S DISEASE: 12,096 (1.74%)
   7. HYPERTENSION: 11,985 (1.73%)
   8. PROPHYLAXIS: 9,765 (1.41%)
   9. PSORIASIS: 9,020 (1.30%)
  10. COLITIS ULCERATIVE: 9,019 (1.30%)
  11. DIABETES MELLITUS: 8,661 (1.25%)
  12. PULMONARY ARTERIAL HYPERTENSION: 7,489 (1.08%)
  13. PROSTATE CANCER: 7,431 (1.07%)
  14. MIGRAINE: 7,394 (1.07%)
  15. DIFFUSE LARGE B-CELL LYMPHOMA: 7,163 (1.03%)
  16. PAIN: 7,057 (1.02%)
  17. PSORIATIC ARTHROPATHY: 6,885 (0.99%)
  18. WEIGHT CONTROL: 6,588 (0.95%)
  19. MULTIPLE SCLEROSIS: 6,554 (0.95%)
  20. ACUTE MYELOID LEUKAEMIA: 5,541 (0.80%)
  21. DEPRESSION: 5,273 (0.76%)
  22. BREAST CANCER: 5,134 (0.74

In [162]:
# Identify disease areas
print("\n" + "-"*40)
print("Disease Area Analysis:")
disease_keywords = {
    'Cancer': ['CANCER', 'CARCINOMA', 'TUMOUR', 'TUMOR', 'LYMPHOMA', 'LEUKEMIA', 'MELANOMA', 'METASTATIC'],
    'Diabetes': ['DIABETES', 'DIABETIC', 'GLUCOSE', 'HYPERGLYCAEMIA'],
    'Cardiovascular': ['HYPERTENSION', 'CARDIAC', 'HEART', 'ATRIAL FIBRILLATION', 'MYOCARDIAL'],
    'Psychiatric': ['DEPRESSION', 'ANXIETY', 'PSYCHOTIC', 'BIPOLAR', 'SCHIZOPHRENIA'],
    'Autoimmune': ['RHEUMATOID ARTHRITIS', 'LUPUS', 'CROHN', 'COLITIS', 'PSORIASIS'],
    'Infection': ['COVID', 'INFECTION', 'HEPATITIS', 'HIV', 'SEPSIS']
}

for area, keywords in disease_keywords.items():
    area_count = indi_clean[indi_clean['indi_pt_clean'].str.contains('|'.join(keywords))]['primaryid'].nunique()
    print(f"  {area}: {area_count:,} patients ({(area_count/406184*100):.1f}%)")


----------------------------------------
Disease Area Analysis:
  Cancer: 42,258 patients (10.4%)
  Diabetes: 13,923 patients (3.4%)
  Cardiovascular: 16,602 patients (4.1%)
  Psychiatric: 9,817 patients (2.4%)
  Autoimmune: 39,023 patients (9.6%)
  Infection: 15,381 patients (3.8%)


##Final Data Merge

In [163]:
# Start with demographics as base

merged_data = demo_processed.copy()
print(f"Base: {merged_data.shape[0]:,} patients from demographics")

Base: 406,184 patients from demographics


In [164]:
# 1. Merge drug data
print("\n1. Merging drug profiles...")
merged_data = merged_data.merge(
    patient_suspect_drugs[['suspect_drugs', 'num_suspect_drugs', 'polypharmacy_category']],
    left_on='primaryid',
    right_index=True,
    how='left'
)
# Also merge concomitant drugs if exists
if 'patient_concom_drugs' in locals():
    merged_data = merged_data.merge(
        patient_concom_drugs[['concomitant_drugs', 'num_concom_drugs']],
        left_on='primaryid',
        right_index=True,
        how='left'
    )
print(f"After drug merge: {merged_data.shape}")


1. Merging drug profiles...
After drug merge: (406184, 12)


In [165]:
# 2. Merge reactions
print("2. Merging reaction profiles...")
merged_data = merged_data.merge(
    patient_reactions[['all_reactions', 'num_reactions', 'has_serious_reaction', 'reaction_severity']],
    left_on='primaryid',
    right_index=True,
    how='left'
)

2. Merging reaction profiles...


In [166]:
print(f"After reaction merge: {merged_data.shape}")

After reaction merge: (406184, 16)


In [167]:
# 3. Merge outcomes
print("3. Merging outcome data...")
merged_data = merged_data.merge(
    patient_outcomes[['outcome_codes', 'is_serious_outcome', 'outcome_descriptions']],
    left_on='primaryid',
    right_index=True,
    how='left'
)
print(f"After outcome merge: {merged_data.shape}")

3. Merging outcome data...
After outcome merge: (406184, 19)


In [168]:
# 4. Merge indications
print("4. Merging indication data...")
merged_data = merged_data.merge(
    patient_indications[['indications', 'num_indications']],
    left_on='primaryid',
    right_index=True,
    how='left'
)
print(f"After indication merge: {merged_data.shape}")

4. Merging indication data...
After indication merge: (406184, 21)


In [169]:
# 5. Quality check - remove patients with no drugs or no reactions
print("\n5. Data quality filtering...")
before_filter = len(merged_data)

# Must have at least drugs and reactions for meaningful analysis
has_drugs = merged_data['suspect_drugs'].notna()
has_reactions = merged_data['all_reactions'].notna()
valid_cases = has_drugs & has_reactions

merged_data = merged_data[valid_cases].copy()
print(f"Removed {before_filter - len(merged_data):,} incomplete cases")
print(f"Final dataset: {merged_data.shape[0]:,} complete cases with {merged_data.shape[1]} features")


5. Data quality filtering...
Removed 370 incomplete cases
Final dataset: 405,814 complete cases with 21 features


In [170]:
# 6. Dataset summary

print("FINAL MERGED DATASET SUMMARY")
for col in merged_data.columns:
    if col != 'primaryid':
        non_null = merged_data[col].notna().sum()
        pct = (non_null / len(merged_data)) * 100
        print(f"  {col}: {non_null:,} ({pct:.1f}%)")

print("\nKey Statistics:")
print(f"  Total cases: {len(merged_data):,}")
print(f"  Cases with serious reactions: {merged_data['has_serious_reaction'].sum():,}")
print(f"  Cases with serious outcomes: {merged_data['is_serious_outcome'].fillna(False).sum():,}")
print(f"  Cases with both serious reactions AND outcomes: {(merged_data['has_serious_reaction'] & merged_data['is_serious_outcome'].fillna(False)).sum():,}")
print(f"  Elderly patients (≥65): {merged_data['is_elderly'].sum():,}")
print(f"  Polypharmacy cases (≥5 drugs): {(merged_data['num_suspect_drugs'] >= 5).sum():,}")

FINAL MERGED DATASET SUMMARY
  age_years: 248,233 (61.2%)
  age_group: 248,230 (61.2%)
  sex_clean: 405,814 (100.0%)
  occr_country: 368,173 (90.7%)
  is_elderly: 405,814 (100.0%)
  is_pediatric: 405,814 (100.0%)
  suspect_drugs: 405,814 (100.0%)
  num_suspect_drugs: 405,814 (100.0%)
  polypharmacy_category: 405,777 (100.0%)
  concomitant_drugs: 109,890 (27.1%)
  num_concom_drugs: 109,890 (27.1%)
  all_reactions: 405,814 (100.0%)
  num_reactions: 405,814 (100.0%)
  has_serious_reaction: 405,814 (100.0%)
  reaction_severity: 405,677 (100.0%)
  outcome_codes: 222,355 (54.8%)
  is_serious_outcome: 222,355 (54.8%)
  outcome_descriptions: 222,355 (54.8%)
  indications: 290,333 (71.5%)
  num_indications: 290,333 (71.5%)

Key Statistics:
  Total cases: 405,814
  Cases with serious reactions: 48,589
  Cases with serious outcomes: 115,725
  Cases with both serious reactions AND outcomes: 35,069
  Elderly patients (≥65): 98,528
  Polypharmacy cases (≥5 drugs): 16,506


In [171]:
merged_data.head()

,primaryid,age_years,age_group,sex_clean,occr_country,is_elderly,is_pediatric,suspect_drugs,num_suspect_drugs,polypharmacy_category,...,num_concom_drugs,all_reactions,num_reactions,has_serious_reaction,reaction_severity,outcome_codes,is_serious_outcome,outcome_descriptions,indications,num_indications
0,1001678125,56.0,middle_age,Female,CA,0,0,"[SANDOSTATIN LAR DEPOT, SANDOSTATIN]",2,dual_therapy,...,11.0,"[BLOOD CREATINE INCREASED, FALL, SINUSITIS, SK...",76.0,True,extreme,[OT],False,[Other Serious (Important Medical Event)],[NEUROENDOCRINE TUMOUR],1.0
1,1002872124,57.0,middle_age,Female,CA,0,0,"[SANDOSTATIN LAR DEPOT, AFINITOR, SANDOSTATIN]",3,moderate_poly,...,4.0,"[PNEUMONITIS, ABDOMINAL PAIN UPPER, CARCINOID ...",24.0,True,extreme,"[OT, HO]",True,"[Other Serious (Important Medical Event), Hosp...","[PANCREATIC NEUROENDOCRINE TUMOUR, CARCINOID T...",2.0
2,100293663,32.0,adult,Male,AU,0,0,"[CYCLOSPORINE, BASILIXIMAB, MYCOPHENOLATE MOFE...",4,moderate_poly,...,NaN,"[STAPHYLOCOCCAL INFECTION, MYCOBACTERIUM HAEMO...",4.0,False,moderate,[OT],False,[Other Serious (Important Medical Event)],"[RENAL TRANSPLANT, IMMUNOSUPPRESSANT DRUG THER...",2.0
3,1005450710,68.0,elderly,Female,US,1,0,"[ENBREL, METHOTREXATE SODIUM]",2,dual_therapy,...,7.0,"[DRUG HYPERSENSITIVITY, DRUG ERUPTION]",2.0,False,few,NaN,NaN,NaN,NaN,NaN
4,1005762118,57.0,middle_age,Male,CA,0,0,"[EXELON, XOLAIR]",2,dual_therapy,...,9.0,"[PHOTOSENSITIVITY REACTION, PARKINSON'S DISEAS...",32.0,False,extreme,[HO],True,[Hospitalization - Initial or Prolonged],"[ANTIBIOTIC PROPHYLAXIS, ASTHMA]",2.0


In [172]:
merged_data.shape

(405814, 21)

In [173]:
merged_data.memory_usage(deep=True).sum() / (1024**2)   # in MB

np.float64(259.0941114425659)

##Save the processed Q1 file to drive

In [174]:
output_path = '/content/drive/MyDrive/Dataset/processed_quarters/'
os.makedirs(output_path, exist_ok=True)

In [175]:
print(f"Saving Q1 processed data...")
print(f"Q1 shape: {merged_data.shape}")
merged_data.to_pickle(f'{output_path}processed_2024Q1.pkl')
print(f"✓ Saved Q1 to: {output_path}processed_2024Q1.pkl")

Saving Q1 processed data...
Q1 shape: (405814, 21)
✓ Saved Q1 to: /content/drive/MyDrive/Dataset/processed_quarters/processed_2024Q1.pkl


###Now we got the logic to process a quarter data towards our goal. So now we are creating a pipeline to process all other quarter data.

In [122]:
import pandas as pd
import numpy as np
import zipfile
import os
import warnings
import json
from datetime import datetime
warnings.filterwarnings('ignore')

def process_quarter(quarter_name, zip_path, output_path, verbose=True):
    """
    Complete pipeline to process one quarter of FAERS data with quality checks

    Parameters:
    -----------
    quarter_name: str, e.g., '2024Q1', '2024Q2'
    zip_path: str, path to the zip file
    output_path: str, where to save processed data
    verbose: bool, whether to print progress messages

    Returns:
    --------
    tuple: (merged_data, quality_report)
    """

    # Initialize quality report
    quality_report = {
        'quarter': quarter_name,
        'processing_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'issues_found': [],
        'data_quality_metrics': {},
        'processing_steps': []
    }

    def log_step(message, data=None):
        """Helper function to log steps"""
        if verbose:
            print(message)
        quality_report['processing_steps'].append({
            'step': message,
            'timestamp': datetime.now().strftime('%H:%M:%S'),
            'data': data
        })

    try:
        # ============= STEP 1: EXTRACT ZIP =============
        extract_path = f'/content/{quarter_name}/'
        os.makedirs(extract_path, exist_ok=True)

        log_step(f"\n{'='*60}")
        log_step(f"PROCESSING {quarter_name}")
        log_step(f"{'='*60}")
        log_step(f"1. Extracting {zip_path}...")

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

        # Find ASCII folder
        ascii_folder = None
        for root, dirs, files in os.walk(extract_path):
            if any(file.endswith('.txt') for file in files):
                ascii_folder = root
                break

        if not ascii_folder:
            raise ValueError(f"No ASCII files found in {extract_path}")

        log_step(f"   ASCII files found in: {ascii_folder}")

        # ============= STEP 2: LOAD DATA FILES =============
        log_step(f"\n2. Loading data files...")

        # Determine file suffix (24Q1, 24Q2, etc.)
        year_suffix = quarter_name.replace('20', '')  # 2024Q1 -> 24Q1

        # Load all required files with error handling
        data_files = {}
        file_sizes = {}

        for file_type in ['DEMO', 'DRUG', 'REAC', 'OUTC', 'INDI']:
            file_name = f'{file_type}{year_suffix}.txt'
            file_path = os.path.join(ascii_folder, file_name)

            if os.path.exists(file_path):
                try:
                    data_files[file_type] = pd.read_csv(
                        file_path,
                        sep='$',
                        encoding='latin-1',
                        low_memory=False
                    )
                    file_sizes[file_type] = data_files[file_type].shape[0]
                    log_step(f"   {file_type}: {file_sizes[file_type]:,} records")
                except Exception as e:
                    quality_report['issues_found'].append(f"Failed to load {file_type}: {str(e)}")
                    raise
            else:
                quality_report['issues_found'].append(f"Missing file: {file_name}")
                raise FileNotFoundError(f"Required file {file_name} not found")

        # Unpack loaded dataframes
        demo = data_files['DEMO']
        drug = data_files['DRUG']
        reac = data_files['REAC']
        outc = data_files['OUTC']
        indi = data_files['INDI']

        # ============= STEP 3: DATA QUALITY CHECKS =============
        log_step(f"\n3. Running data quality checks...")

        # Check for missing primary IDs
        missing_ids = {}
        for name, df in data_files.items():
            missing_count = df['primaryid'].isna().sum()
            missing_ids[name] = missing_count
            if missing_count > 0:
                quality_report['issues_found'].append(f"{name} has {missing_count} missing primary IDs")

        quality_report['data_quality_metrics']['missing_primary_ids'] = missing_ids

        # ============= STEP 4: HANDLE DUPLICATES =============
        log_step(f"\n4. Handling duplicates...")

        # Check and remove DRUG duplicates
        drug_dup_count = drug[['primaryid', 'drug_seq']].duplicated().sum()
        if drug_dup_count > 0:
            drug = drug.drop_duplicates(subset=['primaryid', 'drug_seq'], keep='first')
            log_step(f"   Removed {drug_dup_count} duplicate drug records")
            quality_report['data_quality_metrics']['drug_duplicates_removed'] = drug_dup_count

        # Check and remove REAC duplicates
        reac_dup_count = reac[['primaryid', 'pt']].duplicated().sum()
        if reac_dup_count > 0:
            reac = reac.drop_duplicates(subset=['primaryid', 'pt'], keep='first')
            log_step(f"   Removed {reac_dup_count} duplicate reaction records")
            quality_report['data_quality_metrics']['reac_duplicates_removed'] = reac_dup_count

        # ============= STEP 5: PROCESS DEMOGRAPHICS =============
        log_step(f"\n5. Processing demographics...")
        demo_clean = demo.copy()

        # Standardize age to years
        age_conversion = {
            'YR': 1, 'YRS': 1, 'MON': 1/12, 'WK': 1/52,
            'DY': 1/365, 'DEC': 10, 'HR': 1/(365*24)
        }

        demo_clean['age_years'] = np.nan
        age_converted_counts = {}

        for unit, multiplier in age_conversion.items():
            mask = demo_clean['age_cod'] == unit
            if mask.any():
                count = mask.sum()
                demo_clean.loc[mask, 'age_years'] = pd.to_numeric(
                    demo_clean.loc[mask, 'age'], errors='coerce'
                ) * multiplier
                age_converted_counts[unit] = count

        quality_report['data_quality_metrics']['age_conversions'] = age_converted_counts

        # Create age groups
        demo_clean['age_group'] = pd.cut(
            demo_clean['age_years'],
            bins=[0, 2, 12, 18, 45, 65, 80, 150],
            labels=['infant', 'child', 'adolescent', 'adult', 'middle_age', 'elderly', 'very_elderly'],
            include_lowest=True
        )

        # Clean sex values
        sex_mapping = {
            'M': 'Male', 'F': 'Female',
            'UNK': 'Unknown', 'U': 'Unknown',
            np.nan: 'Unknown'
        }
        demo_clean['sex_clean'] = demo_clean['sex'].map(sex_mapping).fillna('Unknown')

        # Create binary features
        demo_clean['is_elderly'] = (demo_clean['age_years'] >= 65).astype(int)
        demo_clean['is_pediatric'] = (demo_clean['age_years'] < 18).astype(int)
        #demo_clean['is_female'] = (demo_clean['sex_clean'] == 'Female').astype(int)

        # Select final columns
        demo_processed = demo_clean[[
            'primaryid', 'age_years', 'age_group', 'sex_clean',
            'occr_country', 'is_elderly', 'is_pediatric',
        ]].copy()

        # ============= STEP 6: PROCESS DRUGS =============
        log_step(f"\n6. Processing drugs...")

        # Process suspect drugs
        suspect_drugs = drug[drug['role_cod'].isin(['PS', 'SS', 'I'])].copy()
        suspect_drugs['drugname_clean'] = suspect_drugs['drugname'].str.upper().str.strip()

        # Remove invalid drug names
        invalid_names = [
            'UNKNOWN', 'UNK', 'NULL', 'NA', 'N/A',
            'NOT SPECIFIED', 'UNSPECIFIED', 'BLINDED'
        ]
        invalid_drug_count = suspect_drugs['drugname_clean'].isin(invalid_names).sum()
        suspect_drugs = suspect_drugs[~suspect_drugs['drugname_clean'].isin(invalid_names)]

        if invalid_drug_count > 0:
            quality_report['data_quality_metrics']['invalid_drugs_removed'] = invalid_drug_count

        # Process concomitant drugs
        concomitant_drugs = drug[drug['role_cod'] == 'C'].copy()
        concomitant_drugs['drugname_clean'] = concomitant_drugs['drugname'].str.upper().str.strip()
        concomitant_drugs = concomitant_drugs[~concomitant_drugs['drugname_clean'].isin(invalid_names)]

        # Create patient drug profiles
        patient_suspect_drugs = suspect_drugs.groupby('primaryid')['drugname_clean'].apply(
            lambda x: list(set(x))
        ).to_frame()
        patient_suspect_drugs.columns = ['suspect_drugs']
        patient_suspect_drugs['num_suspect_drugs'] = patient_suspect_drugs['suspect_drugs'].apply(len)

        # Polypharmacy categories
        patient_suspect_drugs['polypharmacy_category'] = pd.cut(
            patient_suspect_drugs['num_suspect_drugs'],
            bins=[0, 1, 2, 5, 10, 1000],
            labels=['monotherapy', 'dual_therapy', 'moderate_poly', 'severe_poly', 'extreme_poly']
        )

        # Concomitant drugs
        patient_concom_drugs = concomitant_drugs.groupby('primaryid')['drugname_clean'].apply(
            lambda x: list(set(x))
        ).to_frame()
        patient_concom_drugs.columns = ['concomitant_drugs']
        patient_concom_drugs['num_concom_drugs'] = patient_concom_drugs['concomitant_drugs'].apply(len)

        patient_suspect_drugs['is_interaction_case'] = (patient_suspect_drugs['num_suspect_drugs'] > 1).astype(int)

        primary_drug = suspect_drugs[suspect_drugs['role_cod'] == 'PS'].groupby('primaryid')['drugname_clean'].first()
        patient_suspect_drugs['primary_drug'] = patient_suspect_drugs.index.map(primary_drug)

        # ============= STEP 7: PROCESS REACTIONS =============
        log_step(f"\n7. Processing reactions...")
        reac_clean = reac.copy()
        reac_clean['pt_clean'] = reac_clean['pt'].str.upper().str.strip()

        # Remove invalid reactions
        invalid_reactions = ['UNKNOWN', 'NA', 'NONE', 'NO ADVERSE EVENT', 'NO ADVERSE DRUG REACTION']
        invalid_reac_count = reac_clean['pt_clean'].isin(invalid_reactions).sum()
        reac_clean = reac_clean[~reac_clean['pt_clean'].isin(invalid_reactions)]

        if invalid_reac_count > 0:
            quality_report['data_quality_metrics']['invalid_reactions_removed'] = invalid_reac_count

        # Identify serious reactions
        serious_keywords = [
            'DEATH', 'DIED', 'FATAL', 'CARDIAC ARREST', 'MYOCARDIAL INFARCTION',
            'STROKE', 'ANAPHYLACTIC', 'HEPATIC FAILURE', 'RENAL FAILURE',
            'RESPIRATORY FAILURE', 'SEPSIS', 'SEPTIC SHOCK', 'SUICIDE'
        ]

        reac_clean['is_serious'] = reac_clean['pt_clean'].apply(
            lambda x: any(keyword in x for keyword in serious_keywords) if pd.notna(x) else False
        )

        # Patient reaction profiles
        patient_reactions = reac_clean.groupby('primaryid').agg({
            'pt_clean': lambda x: list(set(x)),
            'is_serious': 'max'
        }).rename(columns={'pt_clean': 'all_reactions', 'is_serious': 'has_serious_reaction'})

        patient_reactions['num_reactions'] = patient_reactions['all_reactions'].apply(len)
        patient_reactions['reaction_severity'] = pd.cut(
            patient_reactions['num_reactions'],
            bins=[0, 1, 3, 5, 10, 1000],
            labels=['single', 'few', 'moderate', 'many', 'extreme']
        )

        # ============= STEP 8: PROCESS OUTCOMES =============
        log_step(f"\n8. Processing outcomes...")

        serious_outcome_codes = ['DE', 'LT', 'HO', 'DS', 'CA']
        outc['is_serious_outcome'] = outc['outc_cod'].isin(serious_outcome_codes)

        outcome_mapping = {
            'DE': 'Death', 'LT': 'Life-Threatening', 'HO': 'Hospitalization',
            'DS': 'Disability', 'CA': 'Congenital Anomaly',
            'RI': 'Required Intervention', 'OT': 'Other Serious'
        }
        outc['outcome_desc'] = outc['outc_cod'].map(outcome_mapping)

        patient_outcomes = outc.groupby('primaryid').agg({
            'outc_cod': lambda x: list(x),
            'is_serious_outcome': 'max',
            'outcome_desc': lambda x: list(x)
        }).rename(columns={'outc_cod': 'outcome_codes', 'outcome_desc': 'outcome_descriptions'})

        # ============= STEP 9: PROCESS INDICATIONS =============
        log_step(f"\n9. Processing indications...")

        indi_clean = indi.copy()
        indi_clean['indi_pt_clean'] = indi_clean['indi_pt'].str.upper().str.strip()

        invalid_indications = ['UNKNOWN', 'UNK', 'PRODUCT USED FOR UNKNOWN INDICATION', 'NA', 'NONE']
        invalid_indi_count = indi_clean['indi_pt_clean'].isin(invalid_indications).sum()
        indi_clean = indi_clean[~indi_clean['indi_pt_clean'].isin(invalid_indications)]

        if invalid_indi_count > 0:
            quality_report['data_quality_metrics']['invalid_indications_removed'] = invalid_indi_count

        patient_indications = indi_clean.groupby('primaryid')['indi_pt_clean'].apply(
            lambda x: list(set(x))
        ).to_frame()
        patient_indications.columns = ['indications']
        patient_indications['num_indications'] = patient_indications['indications'].apply(len)

        # ============= STEP 10: MERGE ALL DATA =============
        log_step(f"\n10. Merging all datasets...")

        # Start with demographics
        merged = demo_processed.copy()
        initial_count = len(merged)

        # Merge all components
        merge_steps = [
            (patient_suspect_drugs[['suspect_drugs', 'num_suspect_drugs', 'polypharmacy_category']], 'drugs'),
            (patient_concom_drugs[['concomitant_drugs', 'num_concom_drugs']], 'concom_drugs'),
            (patient_reactions[['all_reactions', 'num_reactions', 'has_serious_reaction', 'reaction_severity']], 'reactions'),
            (patient_outcomes[['outcome_codes', 'is_serious_outcome', 'outcome_descriptions']], 'outcomes'),
            (patient_indications[['indications', 'num_indications']], 'indications')
        ]

        for df_to_merge, name in merge_steps:
            before_merge = len(merged)
            merged = merged.merge(df_to_merge, left_on='primaryid', right_index=True, how='left')
            after_merge = len(merged)

            if before_merge != after_merge:
                quality_report['issues_found'].append(f"Row count changed during {name} merge: {before_merge} -> {after_merge}")

        # ============= STEP 11: QUALITY FILTERING =============
        log_step(f"\n11. Quality filtering...")

        # Must have drugs and reactions for meaningful analysis
        before_filter = len(merged)
        has_drugs = merged['suspect_drugs'].notna()
        has_reactions = merged['all_reactions'].notna()
        valid_cases = has_drugs & has_reactions

        merged = merged[valid_cases].copy()
        filtered_count = before_filter - len(merged)

        if filtered_count > 0:
            log_step(f"   Removed {filtered_count:,} incomplete cases")
            quality_report['data_quality_metrics']['incomplete_cases_removed'] = filtered_count

        # ============= STEP 11.5: MISSING VALUE REPORT =============
        if verbose:
            log_step(f"\n11.5. Missing Value Analysis...")

            # Calculate missing values for each column
            missing_report = {}
            total_rows = len(merged)

            print(f"\n{'='*60}")
            print(f"MISSING VALUES REPORT FOR {quarter_name}")
            print(f"{'='*60}")
            print(f"Total cases: {total_rows:,}")
            print(f"\nColumns with missing values:")
            print("-" * 40)

            has_missing = False
            for col in merged.columns:
                missing_count = merged[col].isna().sum()
                missing_pct = (missing_count / total_rows) * 100

                missing_report[col] = {
                    'missing_count': missing_count,
                    'missing_pct': missing_pct
                }

                # Only show columns that have missing values
                if missing_count > 0:
                    has_missing = True
                    print(f"  {col:30s}: {missing_count:8,} ({missing_pct:5.1f}%)")

            if not has_missing:
                print("No missing values in any column!")

            # Add to quality report
            quality_report['data_quality_metrics']['missing_values'] = missing_report

            # Show summary of completeness
            print(f"\n{'='*60}")
            print("DATA COMPLETENESS SUMMARY:")
            print("-" * 40)

            # Categorize columns by completeness
            complete_cols = []
            partial_cols = []
            sparse_cols = []

            for col, stats in missing_report.items():
                if stats['missing_pct'] == 0:
                    complete_cols.append(col)
                elif stats['missing_pct'] < 30:
                    partial_cols.append(f"{col} ({100-stats['missing_pct']:.1f}% complete)")
                else:
                    sparse_cols.append(f"{col} ({100-stats['missing_pct']:.1f}% complete)")

            print(f"Fully Complete Columns ({len(complete_cols)}):")
            if complete_cols:
                for col in complete_cols[:5]:  # Show first 5
                    print(f" {col}")
                if len(complete_cols) > 5:
                    print(f"  ... and {len(complete_cols)-5} more")

            if partial_cols:
                print(f"\nPartially Complete Columns (>70% complete):")
                for col in partial_cols:
                    print(f" {col}")

            if sparse_cols:
                print(f"\nSparse Columns (<70% complete):")
                for col in sparse_cols:
                    print(f" {col}")

            print(f"{'='*60}\n")

        # ============= STEP 12: CALCULATE FINAL METRICS =============
        final_metrics = {
            'total_cases': len(merged),
            'cases_with_serious_reactions': merged['has_serious_reaction'].sum(),
            'cases_with_serious_outcomes': merged['is_serious_outcome'].fillna(False).sum(),
            'elderly_patients': merged['is_elderly'].sum(),
            'pediatric_patients': merged['is_pediatric'].sum(),
            'polypharmacy_cases': (merged['num_suspect_drugs'] >= 5).sum(),
            'average_drugs_per_patient': merged['num_suspect_drugs'].mean(),
            'average_reactions_per_patient': merged['num_reactions'].mean(),
            'missing_age_pct': (merged['age_years'].isna().sum() / len(merged)) * 100,
            'missing_country_pct': (merged['occr_country'].isna().sum() / len(merged)) * 100
        }

        quality_report['data_quality_metrics']['final_metrics'] = final_metrics

        log_step(f"\n✓ {quarter_name} processed successfully!")
        log_step(f"  Final shape: {merged.shape[0]:,} cases, {merged.shape[1]} features")

        # ============= STEP 13: SAVE PROCESSED DATA =============
        log_step(f"\n12. Saving processed data...")

        # Create output directory if it doesn't exist
        os.makedirs(output_path, exist_ok=True)

        # Save processed data
        pickle_path = f'{output_path}processed_{quarter_name}.pkl'
        merged.to_pickle(pickle_path)
        log_step(f"   Saved to: {pickle_path}")

        # Save quality report
        report_path = f'{output_path}quality_report_{quarter_name}.json'
        with open(report_path, 'w') as f:
            json.dump(quality_report, f, indent=2, default=str)
        log_step(f"   Quality report saved to: {report_path}")

        # ============= STEP 14: CLEANUP =============
        log_step(f"\n13. Cleaning up temporary files...")
        import shutil
        if os.path.exists(extract_path):
            shutil.rmtree(extract_path)
            log_step(f"   Removed temporary extraction folder")

        return merged, quality_report

    except Exception as e:
        quality_report['error'] = str(e)
        quality_report['status'] = 'FAILED'

        # Save error report
        error_report_path = f'{output_path}error_report_{quarter_name}.json'
        with open(error_report_path, 'w') as f:
            json.dump(quality_report, f, indent=2, default=str)

        log_step(f"\nError processing {quarter_name}: {str(e)}")
        raise

##Process all quarters

In [123]:
# Process all quarters

dataset_path = '/content/drive/MyDrive/Dataset/'
output_path = '/content/drive/MyDrive/Dataset/processed_quarters/'
os.makedirs(output_path, exist_ok=True)

# Define quarters to process
quarters_to_process = [
    ('2024Q2', f'{dataset_path}faers_ascii_2024q2.zip'),
    ('2024Q3', f'{dataset_path}faers_ascii_2024q3.zip'),
    ('2024Q4', f'{dataset_path}faers_ascii_2024Q4.zip')  # Note: Q4 might be uppercase
]

# Store all processed data and reports
all_processed_data = {}
all_quality_reports = {}

for quarter_name, zip_path in quarters_to_process:
    try:
        if os.path.exists(zip_path):
            print(f"\n{'='*60}")
            print(f"Processing {quarter_name}...")
            print(f"{'='*60}")

            processed_data, quality_report = process_quarter(
                quarter_name=quarter_name,
                zip_path=zip_path,
                output_path=output_path,
                verbose=True
            )

            all_processed_data[quarter_name] = processed_data
            all_quality_reports[quarter_name] = quality_report

            print(f"{quarter_name} completed successfully!")
            print(f"   Cases: {len(processed_data):,}")
            print(f"   Features: {processed_data.shape[1]}")

        else:
            print(f"Warning: {zip_path} not found, skipping {quarter_name}")

    except Exception as e:
        print(f"Failed to process {quarter_name}: {str(e)}")
        continue


Processing 2024Q2...

PROCESSING 2024Q2
1. Extracting /content/drive/MyDrive/Dataset/faers_ascii_2024q2.zip...
   ASCII files found in: /content/2024Q2/ASCII

2. Loading data files...
   DEMO: 397,119 records
   DRUG: 1,888,937 records
   REAC: 1,445,044 records
   OUTC: 291,572 records
   INDI: 1,187,626 records

3. Running data quality checks...

4. Handling duplicates...
   Removed 1249 duplicate drug records
   Removed 21798 duplicate reaction records

5. Processing demographics...

6. Processing drugs...

7. Processing reactions...

8. Processing outcomes...

9. Processing indications...

10. Merging all datasets...

11. Quality filtering...
   Removed 563 incomplete cases

11.5. Missing Value Analysis...

MISSING VALUES REPORT FOR 2024Q2
Total cases: 396,556

Columns with missing values:
----------------------------------------
  age_years                     :  161,045 ( 40.6%)
  age_group                     :  161,047 ( 40.6%)
  occr_country                  :   38,260 (  9.6

##Validation & Quality Assurance

In [176]:
def validate_processed_quarters(output_path):
    """
    Validate that all quarters were processed consistently
    """

    print("\n" + "="*60)
    print("VALIDATING PROCESSED QUARTERS")
    print("="*60)

    # Load all processed quarters
    processed_files = glob.glob(f'{output_path}processed_*.pkl')

    if not processed_files:
        print("No processed files found!")
        return

    quarters_data = {}
    for file_path in processed_files:
        quarter_name = os.path.basename(file_path).replace('processed_', '').replace('.pkl', '')
        quarters_data[quarter_name] = pd.read_pickle(file_path)

    print(f"\nFound {len(quarters_data)} processed quarters:")

    # Check consistency
    validation_results = {}

    for quarter_name, data in sorted(quarters_data.items()):
        print(f"\n{quarter_name}:")
        print(f"  Shape: {data.shape}")
        print(f"  Columns: {list(data.columns)}")

        validation_results[quarter_name] = {
            'shape': data.shape,
            'columns': list(data.columns),
            'missing_values': data.isnull().sum().to_dict(),
            'serious_reaction_rate': (data['has_serious_reaction'].sum() / len(data)) * 100,
            'elderly_rate': (data['is_elderly'].sum() / len(data)) * 100,
            'avg_drugs': data['num_suspect_drugs'].mean(),
            'avg_reactions': data['num_reactions'].mean()
        }

    # Check if all quarters have same columns
    all_columns = [set(v['columns']) for v in validation_results.values()]
    if len(set(map(tuple, all_columns))) == 1:
        print("\nAll quarters have consistent columns")
    else:
        print("\nWarning: Quarters have different columns!")

    # Compare key metrics across quarters
    print("\nKey Metrics Comparison:")
    print("-" * 40)

    metrics_df = pd.DataFrame({
        quarter: {
            'Cases': results['shape'][0],
            'Serious Reactions %': f"{results['serious_reaction_rate']:.1f}",
            'Elderly %': f"{results['elderly_rate']:.1f}",
            'Avg Drugs': f"{results['avg_drugs']:.2f}",
            'Avg Reactions': f"{results['avg_reactions']:.2f}"
        }
        for quarter, results in validation_results.items()
    }).T

    print(metrics_df)

    return validation_results

# Run validation
validation_results = validate_processed_quarters(output_path)


VALIDATING PROCESSED QUARTERS

Found 4 processed quarters:

2024Q1:
  Shape: (405814, 21)
  Columns: ['primaryid', 'age_years', 'age_group', 'sex_clean', 'occr_country', 'is_elderly', 'is_pediatric', 'suspect_drugs', 'num_suspect_drugs', 'polypharmacy_category', 'concomitant_drugs', 'num_concom_drugs', 'all_reactions', 'num_reactions', 'has_serious_reaction', 'reaction_severity', 'outcome_codes', 'is_serious_outcome', 'outcome_descriptions', 'indications', 'num_indications']

2024Q2:
  Shape: (396556, 21)
  Columns: ['primaryid', 'age_years', 'age_group', 'sex_clean', 'occr_country', 'is_elderly', 'is_pediatric', 'suspect_drugs', 'num_suspect_drugs', 'polypharmacy_category', 'concomitant_drugs', 'num_concom_drugs', 'all_reactions', 'num_reactions', 'has_serious_reaction', 'reaction_severity', 'outcome_codes', 'is_serious_outcome', 'outcome_descriptions', 'indications', 'num_indications']

2024Q3:
  Shape: (405168, 21)
  Columns: ['primaryid', 'age_years', 'age_group', 'sex_clean', 'oc

##Now we are processing 2025 Dataset

In [178]:
# Process 2025 quarters with CORRECT quarter names
quarters_to_process = [
    ('2025Q1', f'{dataset_path}faers_ascii_2025q1.zip'),  # Changed from 2024Q2 to 2025Q1
    ('2025Q2', f'{dataset_path}faers_ascii_2025q2.zip'),  # Fixed: was '205q2' (missing '2')
    ('2025Q3', f'{dataset_path}faers_ascii_2025q3.zip')   # Changed from 2024Q3 to 2025Q3
]

# Store all processed data and reports
all_processed_data = {}
all_quality_reports = {}

for quarter_name, zip_path in quarters_to_process:
    try:
        if os.path.exists(zip_path):
            print(f"\n{'='*60}")
            print(f"Processing {quarter_name}...")
            print(f"{'='*60}")

            processed_data, quality_report = process_quarter(
                quarter_name=quarter_name,
                zip_path=zip_path,
                output_path=output_path,
                verbose=True
            )

            all_processed_data[quarter_name] = processed_data
            all_quality_reports[quarter_name] = quality_report

            print(f"{quarter_name} completed successfully!")
            print(f"   Cases: {len(processed_data):,}")
            print(f"   Features: {processed_data.shape[1]}")

        else:
            print(f"Warning: {zip_path} not found, skipping {quarter_name}")

    except Exception as e:
        print(f"Failed to process {quarter_name}: {str(e)}")
        continue


Processing 2025Q1...

PROCESSING 2025Q1
1. Extracting /content/drive/MyDrive/Dataset/faers_ascii_2025q1.zip...
   ASCII files found in: /content/2025Q1/ASCII

2. Loading data files...
   DEMO: 400,514 records
   DRUG: 2,008,162 records
   REAC: 1,432,926 records
   OUTC: 304,027 records
   INDI: 1,205,861 records

3. Running data quality checks...

4. Handling duplicates...
   Removed 46753 duplicate drug records
   Removed 18571 duplicate reaction records

5. Processing demographics...

6. Processing drugs...

7. Processing reactions...

8. Processing outcomes...

9. Processing indications...

10. Merging all datasets...

11. Quality filtering...
   Removed 627 incomplete cases

11.5. Missing Value Analysis...

MISSING VALUES REPORT FOR 2025Q1
Total cases: 399,887

Columns with missing values:
----------------------------------------
  age_years                     :  156,691 ( 39.2%)
  age_group                     :  156,696 ( 39.2%)
  occr_country                  :   16,126 (  4.

###Validation & Quality Assurance

In [179]:
def validate_processed_quarters(output_path):
    """
    Validate that all quarters were processed consistently
    """

    print("\n" + "="*60)
    print("VALIDATING PROCESSED QUARTERS")
    print("="*60)

    # Load all processed quarters
    processed_files = glob.glob(f'{output_path}processed_*.pkl')

    if not processed_files:
        print("No processed files found!")
        return

    quarters_data = {}
    for file_path in processed_files:
        quarter_name = os.path.basename(file_path).replace('processed_', '').replace('.pkl', '')
        quarters_data[quarter_name] = pd.read_pickle(file_path)

    print(f"\nFound {len(quarters_data)} processed quarters:")

    # Check consistency
    validation_results = {}

    for quarter_name, data in sorted(quarters_data.items()):
        print(f"\n{quarter_name}:")
        print(f"  Shape: {data.shape}")
        print(f"  Columns: {list(data.columns)}")

        validation_results[quarter_name] = {
            'shape': data.shape,
            'columns': list(data.columns),
            'missing_values': data.isnull().sum().to_dict(),
            'serious_reaction_rate': (data['has_serious_reaction'].sum() / len(data)) * 100,
            'elderly_rate': (data['is_elderly'].sum() / len(data)) * 100,
            'avg_drugs': data['num_suspect_drugs'].mean(),
            'avg_reactions': data['num_reactions'].mean()
        }

    # Check if all quarters have same columns
    all_columns = [set(v['columns']) for v in validation_results.values()]
    if len(set(map(tuple, all_columns))) == 1:
        print("\nAll quarters have consistent columns")
    else:
        print("\nWarning: Quarters have different columns!")

    # Compare key metrics across quarters
    print("\nKey Metrics Comparison:")
    print("-" * 40)

    metrics_df = pd.DataFrame({
        quarter: {
            'Cases': results['shape'][0],
            'Serious Reactions %': f"{results['serious_reaction_rate']:.1f}",
            'Elderly %': f"{results['elderly_rate']:.1f}",
            'Avg Drugs': f"{results['avg_drugs']:.2f}",
            'Avg Reactions': f"{results['avg_reactions']:.2f}"
        }
        for quarter, results in validation_results.items()
    }).T

    print(metrics_df)

    return validation_results

# Run validation
validation_results = validate_processed_quarters(output_path)


VALIDATING PROCESSED QUARTERS

Found 7 processed quarters:

2024Q1:
  Shape: (405814, 21)
  Columns: ['primaryid', 'age_years', 'age_group', 'sex_clean', 'occr_country', 'is_elderly', 'is_pediatric', 'suspect_drugs', 'num_suspect_drugs', 'polypharmacy_category', 'concomitant_drugs', 'num_concom_drugs', 'all_reactions', 'num_reactions', 'has_serious_reaction', 'reaction_severity', 'outcome_codes', 'is_serious_outcome', 'outcome_descriptions', 'indications', 'num_indications']

2024Q2:
  Shape: (396556, 21)
  Columns: ['primaryid', 'age_years', 'age_group', 'sex_clean', 'occr_country', 'is_elderly', 'is_pediatric', 'suspect_drugs', 'num_suspect_drugs', 'polypharmacy_category', 'concomitant_drugs', 'num_concom_drugs', 'all_reactions', 'num_reactions', 'has_serious_reaction', 'reaction_severity', 'outcome_codes', 'is_serious_outcome', 'outcome_descriptions', 'indications', 'num_indications']

2024Q3:
  Shape: (405168, 21)
  Columns: ['primaryid', 'age_years', 'age_group', 'sex_clean', 'oc

###Merge all 7 files

In [180]:
# Check what files we have
processed_files = sorted(glob.glob(f'{output_path}processed_*.pkl'))

print("Found processed files:")
print("-" * 40)
for file in processed_files:
    quarter = file.split('processed_')[1].replace('.pkl', '')
    data = pd.read_pickle(file)
    print(f"{quarter}: {data.shape[0]:,} cases, {data.shape[1]} columns")

Found processed files:
----------------------------------------
quarters/: 405,814 cases, 21 columns
quarters/: 396,556 cases, 21 columns
quarters/: 405,168 cases, 21 columns
quarters/: 410,362 cases, 21 columns
quarters/: 399,887 cases, 21 columns
quarters/: 392,343 cases, 21 columns
quarters/: 438,081 cases, 21 columns


In [9]:
# Load all processed quarters

# 3. Set the path to your processed quarters
output_path = '/content/drive/MyDrive/Dataset/processed_quarters/'

all_data = []
quarters = ['2024Q1', '2024Q2', '2024Q3', '2024Q4', '2025Q1', '2025Q2', '2025Q3']

for quarter in quarters:
    file_path = f'{output_path}processed_{quarter}.pkl'
    try:
        print(f"Loading {quarter}...")
        data = pd.read_pickle(file_path)
        all_data.append(data)
        print(f"  Loaded {len(data):,} cases, {data.shape[1]} columns")
    except FileNotFoundError:
        print(f"  {quarter} not found, skipping")
        continue

# Combine all quarters
print("\nCombining all loaded quarters...")
final_dataset = pd.concat(all_data, ignore_index=True)

print("\n" + "="*60)
print("MERGED DATASET OVERVIEW")
print("="*60)

# Display basic info
print(f"Shape: {final_dataset.shape}")
print(f"Total cases: {final_dataset.shape[0]:,}")
print(f"Total features: {final_dataset.shape[1]}")
print(f"Memory usage: {final_dataset.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Check for duplicates
duplicate_primaryids = final_dataset['primaryid'].duplicated().sum()
if duplicate_primaryids > 0:
    print(f"\nFound {duplicate_primaryids:,} duplicate primaryids")
    final_dataset = final_dataset.drop_duplicates(subset=['primaryid'], keep='first')
    print(f"After removing duplicates: {final_dataset.shape[0]:,} cases")

# View first few rows
print("\n" + "="*60)
print("FIRST 5 ROWS OF MERGED DATASET")
print("="*60)
print(final_dataset.head())

# View column names
print("\n" + "="*60)
print("COLUMNS IN FINAL DATASET")
print("="*60)
print(list(final_dataset.columns))

# Quick statistics
print("\n" + "="*60)
print("QUICK STATISTICS")
print("="*60)
print(f"Unique patients: {final_dataset['primaryid'].nunique():,}")
print(f"Elderly patients: {final_dataset['is_elderly'].sum():,}")
print(f"Pediatric patients: {final_dataset['is_pediatric'].sum():,}")
print(f"Cases with serious reactions: {final_dataset['has_serious_reaction'].sum():,}")

Loading 2024Q1...
  Loaded 405,814 cases, 21 columns
Loading 2024Q2...
  Loaded 396,556 cases, 21 columns
Loading 2024Q3...
  Loaded 405,168 cases, 21 columns
Loading 2024Q4...
  Loaded 410,362 cases, 21 columns
Loading 2025Q1...
  Loaded 399,887 cases, 21 columns
Loading 2025Q2...
  Loaded 392,343 cases, 21 columns
Loading 2025Q3...
  Loaded 438,081 cases, 21 columns

Combining all loaded quarters...

MERGED DATASET OVERVIEW
Shape: (2848211, 21)
Total cases: 2,848,211
Total features: 21
Memory usage: 2062.18 MB

Found 132 duplicate primaryids
After removing duplicates: 2,848,079 cases

FIRST 5 ROWS OF MERGED DATASET
    primaryid  age_years   age_group sex_clean occr_country  is_elderly  \
0  1001678125       56.0  middle_age    Female           CA           0   
1  1002872124       57.0  middle_age    Female           CA           0   
2   100293663       32.0       adult      Male           AU           0   
3  1005450710       68.0     elderly    Female           US           1   


In [10]:
# Create directory structure if it doesn't exist
save_directory = '/content/drive/MyDrive/Dataset/final_dataset/'
os.makedirs(save_directory, exist_ok=True)

# Define the save path
pkl_file_path = f'{save_directory}final_merged_all_7quarters.pkl'

# Save the final_dataset to pickle format
print("="*60)
print("SAVING FINAL DATASET TO GOOGLE DRIVE")
print("="*60)

try:
    # Save to pickle
    final_dataset.to_pickle(pkl_file_path)

    # Verify it saved correctly
    file_size_mb = os.path.getsize(pkl_file_path) / (1024**2)

    print(f"Successfully saved!")
    print(f"\nFile Details:")
    print(f"   Path: {pkl_file_path}")
    print(f"   Size: {file_size_mb:.2f} MB")
    print(f"   Cases: {final_dataset.shape[0]:,}")
    print(f"   Features: {final_dataset.shape[1]}")

    # Quick verification - try loading it back
    print(f"\nVerifying saved file...")
    test_load = pd.read_pickle(pkl_file_path)
    print(f"   File can be loaded successfully")
    print(f"   Verified shape: {test_load.shape}")

    print(f"\n✨ Your data is safely stored in Google Drive!")

except Exception as e:
    print(f"Error saving file: {str(e)}")

SAVING FINAL DATASET TO GOOGLE DRIVE
Successfully saved!

File Details:
   Path: /content/drive/MyDrive/Dataset/final_dataset/final_merged_all_7quarters.pkl
   Size: 719.92 MB
   Cases: 2,848,079
   Features: 21

Verifying saved file...
   File can be loaded successfully
   Verified shape: (2848079, 21)

✨ Your data is safely stored in Google Drive!
